#Introduction

Ce projet s'inscrit dans le module SIG, et correspond aux résultats du Sujet 2.


Par : Alix de la Fouchardiere
Encadrant : Frédéric REN


---

Intitulé de l'exercice :

Réaliser une carte des départs de feux de forêt importants (>200 ha) du Canada en 2020. Distinguer les villes comprises dans la zone d'impact de ces feux. La zone d'impact sera définie selon le champ SIZE_HA


Données :
* feux de forêts importants au canada (>200 Ha) : https://cwfis.cfs.nrcan.gc.ca/datamart/download/nfdbpnt
* carte des limites de villes du canada : https://simplemaps.com/data/canada-cities

#Bibliothèque de données

In [368]:
import geopandas as gpd
import pandas as pd
import folium
import fiona
from shapely.geometry import Point
from google.colab import files


In [100]:
!pip install mapclassify

#Les feux de forêts de plus de 200 Ha ayant eu lieu en 2020

In [268]:
#chargement du fichier
fire_data = gpd.read_file("/content/NFDB_point_large_fires (1).zip")
fire_data.head()

,SRC_AGENCY,FIRE_ID,FIRENAME,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,REP_DATE,ATTK_DATE,...,CFS_REF_ID,CFS_NOTE1,CFS_NOTE2,ACQ_DATE,SRC_AGY2,ECOZONE,ECOZ_REF,ECOZ_NAME,ECOZ_NOM,geometry
0,BC,1950-G00026,NaN,59.876,-131.922,1950,6,4,1950-06-04,NaN,...,BC-1950-1950-G00026,NaN,NaN,2020-05-05,BC,12,12,Boreal Cordillera,CordillCre boreale,POINT Z (-1899364.952 1758151.328 0.000)
1,BC,1951-R00097,NaN,59.760,-132.808,1951,7,15,1951-07-15,NaN,...,BC-1951-1951-R00097,NaN,NaN,2020-05-05,BC,12,12,Boreal Cordillera,CordillCre boreale,POINT Z (-1946555.868 1774479.072 0.000)
2,BC,1950-G00035,NaN,57.463,-122.816,1950,6,14,1950-06-14,NaN,...,BC-1950-1950-G00035,NaN,NaN,2020-05-05,BC,4,4,Taiga Plain,Taiga des plaines,POINT Z (-1577604.846 1279663.687 0.000)
3,BC,1953-G00043,NaN,57.684,-121.925,1953,5,1,1953-05-01,NaN,...,BC-1953-1953-G00043,NaN,NaN,2020-05-05,BC,4,4,Taiga Plain,Taiga des plaines,POINT Z (-1520317.188 1279789.665 0.000)
4,BC,1950-R00029,NaN,59.318,-133.228,1950,6,22,1950-06-22,NaN,...,BC-1950-1950-R00029,NaN,NaN,2020-05-05,BC,12,12,Boreal Cordillera,CordillCre boreale,POINT Z (-1992609.431 1747896.544 0.000)


In [308]:
len(fire_data)

19115

In [269]:
#Visualiser les données de feux de forêts
fire_data.columns

Index(['SRC_AGENCY', 'FIRE_ID', 'FIRENAME', 'LATITUDE', 'LONGITUDE', 'YEAR',
       'MONTH', 'DAY', 'REP_DATE', 'ATTK_DATE', 'OUT_DATE', 'DECADE',
       'SIZE_HA', 'CAUSE', 'PROTZONE', 'FIRE_TYPE', 'MORE_INFO', 'CFS_REF_ID',
       'CFS_NOTE1', 'CFS_NOTE2', 'ACQ_DATE', 'SRC_AGY2', 'ECOZONE', 'ECOZ_REF',
       'ECOZ_NAME', 'ECOZ_NOM', 'geometry'],
      dtype='object')

In [310]:
#Lecture du système de coordonnée
fire_data.crs, len(fire_data)

(<Projected CRS: EPSG:3857>
 Name: WGS 84 / Pseudo-Mercator
 Axis Info [cartesian]:
 - X[east]: Easting (metre)
 - Y[north]: Northing (metre)
 Area of Use:
 - name: World between 85.06°S and 85.06°N.
 - bounds: (-180.0, -85.06, 180.0, 85.06)
 Coordinate Operation:
 - name: Popular Visualisation Pseudo-Mercator
 - method: Popular Visualisation Pseudo Mercator
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich,
 19115)

In [311]:
#Sélectionner les données des feux de forêts relatifs à l'année 2020 uniquement

# Lire la colonne 'YEAR' comme un entier
fire_data['YEAR'] = fire_data['YEAR'].astype(int)

# Sélect by attribute
data_2020 = fire_data[fire_data["YEAR"] == 2020]
data_2020 = data_2020.to_crs(3857)
len(data_2020)

80

In [365]:
#Exploration de la carte
data_2020.explore()

#Les villes du Canada

In [369]:
#Chargement du fichier regroupant les villes du Canada
com = gpd.read_file('/content/canadacities.csv')

In [ ]:
com.head()

On remarque ici, qu'il n'y a pas de géométrie associée.

In [366]:
#Associer une geometrie grâce aux longitudes, latitudes
gdf = gpd.GeoDataFrame(com[["city_ascii","lat","lng"]], geometry = gpd.points_from_xy(com[["city_ascii","lat","lng"]]['lng'], com[["city_ascii","lat","lng"]]['lat']), crs = 'EPSG:4326')

In [317]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [367]:
#Changement du système de coordonnées
gdf = gdf.to_crs(3857)

#Sélectionner les villes impactées par les incendies de plus de 200 Ha en 2020

In [371]:
# Créez un buffer avec la surface d'impact des feux autour des points de départ des feux de forêts. Cette surface d'impact correspond à la colonne 'SIZE_HA' de data_2020
Rayon = (data_2020['SIZE_HA']* 10000)**0.5

buffer = data_2020['geometry'].buffer(Rayon*20)

Ici le rayon du buffer a été multiplié par 20 pour augmenter l'impact des feux. Cela permet également d'obtenir plus de résultats.

In [372]:
# Sélectionner les villes impactées par les feux dans un buffer
cities_in_buffer = gpd.clip(gdf, buffer)

In [334]:
len(cities_in_buffer)

221

In [373]:
#Carte du buffer et des villes
m = buffer.explore(name = "fire")
gdf.explore(m = m, name = "cities")
folium.LayerControl().add_to(m)

m

In [374]:
#Villes située dans la zone d'impact des feux de 2020
cities_in_buffer.explore()

In [337]:
cities_in_buffer.head()

,city_ascii,lat,lng,geometry
852,Cap-Saint-Ignace,47.0333,-70.4667,POINT (-7844317.162 5947511.175)
1379,Saint-Joachim,47.0500,-70.8500,POINT (-7886985.923 5950239.167)
722,L'Islet-sur-Mer,47.1000,-70.3500,POINT (-7831326.177 5958411.920)
867,Saint-Ferreol-les-Neiges,47.1167,-70.8500,POINT (-7886985.923 5961143.329)
1360,Saint-Tite-des-Caps,47.1333,-70.7667,POINT (-7877713.009 5963859.232)


#Mise en page d'une carte intéractives

In [375]:
#Les villes restantes
cities_out_buffer = gpd.overlay(gdf, data_2020, how='difference')
len(cities_out_buffer)

1737

In [376]:
from folium import plugins
#Zones incendiées
p = buffer.explore(style_kwds = dict(color="red",opacity=0.1))

#Villes impactées
cities_in_buffer.explore(m = p,
                         tooltip = ["city_ascii"],
                         tooltip_kwds = dict(aliases = ["Villes impactées :"]),
                         popup = True,
                         style_kwds = dict(color = "black"))


cities_out_buffer.explore(m=p,
                          name = "Villes :",
                          tooltip = ["city_ascii"],
                          tooltip_kwds = dict(aliases = ["Villes : "]),
                          popup = True,
                          style_kwds = dict(color="blue"))

folium.LayerControl().add_to(p)
minimap = plugins.MiniMap()
p.add_child(minimap)

p

In [378]:
# Exportation de la carte html
p.save("Carte_Impact_Feux_de_Forêt_Alix_DLF.html")
files.download("Carte_Impact_Feux_de_Forêt_Alix_DLF.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>